In [ ]:
pip install opencv-python numpy matplotlib scikit-image tensorflow fiftyone kaggle


  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow
Note: you may need to restart the kernel to use updated packages.


In [15]:
import fiftyone as fo
import fiftyone.zoo as foz
import os
import cv2
import numpy as np
from tqdm import tqdm

In [12]:
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=[],  # Only images, no labels
    max_samples=5000,
    dataset_name="coco_train_images"
)

Found annotations at '/home/sala/fiftyone/coco-2017/raw/instances_train2017.json'


 100% |████████████████| 5000/5000 [7.7m elapsed, 0s remaining, 10.4 images/s]      
Writing annotations for 5000 downloaded samples to '/home/sala/fiftyone/coco-2017/train/labels.json'
Dataset info written to '/home/sala/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'train'
 100% |███████████████| 5000/5000 [3.3s elapsed, 0s remaining, 2.0K samples/s]       
Dataset 'coco_train_images' created


In [29]:
dataset = foz.load_zoo_dataset(
    "coco-2014",
    split="train",
    label_types=[],
    classes=[],
    max_samples=5000,
    dataset_name="coco_train_images2"
)

 100% |██████|    1.9Gb/1.9Gb [2.5m elapsed, 0s remaining, 8.5Mb/s]          
Extracting annotations to '/home/sala/fiftyone/coco-2014/raw/instances_train2014.json'
 100% |████████████████| 5000/5000 [7.9m elapsed, 0s remaining, 11.6 images/s]      
Writing annotations for 5000 downloaded samples to '/home/sala/fiftyone/coco-2014/train/labels.json'
Dataset info written to '/home/sala/fiftyone/coco-2014/info.json'
Loading 'coco-2014' split 'train'
 100% |███████████████| 5000/5000 [983.0ms elapsed, 0s remaining, 5.1K samples/s]      
Dataset 'coco_train_images2' created


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prasanshasatpathy/soil-types")

print("Path to dataset files:", path)

/home/sala/miniconda3/envs/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 3.08M/3.08M [00:01<00:00, 2.04MB/s]

Extracting files...
Path to dataset files: /home/sala/.cache/kagglehub/datasets/prasanshasatpathy/soil-types/versions/1


In [7]:
output_dir = "/home/sala/iit/dsgp/soil_analysis_automation_system/data/processed_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Resize images

In [20]:
def preprocess_image(image_path, output_path, size=(224, 224)):
    """
    Load an image, resize it, convert to LAB color space, and save.

    Args:
        image_path (str): Path to the input image.
        output_path (str): Path to save the processed image.
        size (tuple): Target dimensions (width, height) for resizing. Default is (224, 224).

    Returns:
        bool: True if the image was processed successfully, False otherwise.
    """
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        return False  # Skip if the image is not found or cannot be loaded

    # Resize the image
    img_resized = cv2.resize(img, size)

    # Convert to LAB color space
    img_lab = cv2.cvtColor(img_resized, cv2.COLOR_BGR2LAB)

    # Save the processed image
    cv2.imwrite(output_path, img_lab)
    return True

In [17]:
def augment_image(image):
    """
    Augments the image to simulate real-world lighting and focus variations.
    """
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        brightness_range=[0.7, 1.3]
    )

    # Reshape the image for augmentation
    image = image.reshape((1,) + image.shape)  # Adding batch dimension

    # Apply augmentation
    augmented_images = datagen.flow(image, batch_size=1)
    
    # Get the first augmented image
    augmented_image = next(augmented_images)[0].astype(np.uint8)
    
    return augmented_image


In [21]:
import os
import cv2

def process_images_in_folder(input_folder, output_folder, target_size=(224, 224)):
    """
    Process all images in the input folder and save them to the output folder.

    Args:
        input_folder (str): Path to the folder containing input images.
        output_folder (str): Path to the folder where processed images will be saved.
        target_size (tuple): Target dimensions (width, height) for resizing. Default is (224, 224).
    """
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all images in the folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):  # Case-insensitive check
            # Construct full paths
            image_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"processed_{filename}")

            # Preprocess the image
            success = preprocess_image(image_path, output_path, target_size)
            
            if success:
                print(f"Processed and saved: {output_path}")
            else:
                print(f"Skipped (invalid image): {image_path}")

In [30]:
input_folder = "/home/sala/fiftyone/coco-2014/train/data/" 
output_folder = '/home/sala/iit/dsgp/preprocessed_data/general_data'  

process_images_in_folder(input_folder, output_folder)

Processed and saved: /home/sala/iit/dsgp/preprocessed_data/general_data/processed_COCO_train2014_000000022838.jpg
Processed and saved: /home/sala/iit/dsgp/preprocessed_data/general_data/processed_COCO_train2014_000000025172.jpg
Processed and saved: /home/sala/iit/dsgp/preprocessed_data/general_data/processed_COCO_train2014_000000012679.jpg
Processed and saved: /home/sala/iit/dsgp/preprocessed_data/general_data/processed_COCO_train2014_000000016468.jpg
Processed and saved: /home/sala/iit/dsgp/preprocessed_data/general_data/processed_COCO_train2014_000000011766.jpg
Processed and saved: /home/sala/iit/dsgp/preprocessed_data/general_data/processed_COCO_train2014_000000018233.jpg
Processed and saved: /home/sala/iit/dsgp/preprocessed_data/general_data/processed_COCO_train2014_000000014472.jpg
Processed and saved: /home/sala/iit/dsgp/preprocessed_data/general_data/processed_COCO_train2014_000000008595.jpg
Processed and saved: /home/sala/iit/dsgp/preprocessed_data/general_data/processed_COCO_t